# 回调函数(Callbacks)实战指南

## 核心概念

回调函数是在训练过程的特定阶段自动调用的对象，可用于：

1. **模型检查点**: 在训练期间的不同时间点保存模型
2. **提前终止**: 当验证指标停止改善时中断训练
3. **动态调整**: 在训练期间动态调整参数(如学习率)
4. **日志记录**: 记录训练和验证指标用于后续分析
5. **自定义行为**: 实现训练过程中的任意自定义操作

## 实现原理

回调函数通过在训练循环的关键节点插入钩子函数来工作：
- `on_epoch_begin/end`: 每个epoch开始/结束时调用
- `on_batch_begin/end`: 每个batch开始/结束时调用
- `on_train_begin/end`: 训练开始/结束时调用

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

## 1. 准备实验数据

使用MNIST数据集进行演示，这是一个手写数字识别任务。

In [ ]:
# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 数据预处理：归一化到[0,1]区间
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 展平图像：28x28 -> 784
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# 转换标签为one-hot编码
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(f'训练集形状: {x_train.shape}, 标签形状: {y_train.shape}')
print(f'测试集形状: {x_test.shape}, 标签形状: {y_test.shape}')

## 2. 构建实验模型

构建一个简单的多层感知器(MLP)用于演示回调函数的效果。

In [ ]:
def create_model():
    """
    创建一个简单的MLP模型
    
    架构:
    - 输入层: 784维(28x28展平)
    - 隐藏层1: 128个神经元 + ReLU激活
    - Dropout: 0.2防止过拟合
    - 隐藏层2: 64个神经元 + ReLU激活
    - 输出层: 10个神经元(10类) + Softmax激活
    """
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=(784,)),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model = create_model()
model.summary()

## 3. 自定义回调函数

通过继承`keras.callbacks.Callback`类可以创建自定义回调函数，实现任意训练过程监控逻辑。

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    """
    自定义回调函数示例
    
    功能:
    1. 在训练开始和结束时打印消息
    2. 在每个epoch结束时显示损失和准确率
    3. 监控验证准确率，达到阈值时自动停止训练
    """
    
    def __init__(self, target_accuracy=0.95):
        super().__init__()
        self.target_accuracy = target_accuracy
    
    def on_train_begin(self, logs=None):
        """训练开始时调用"""
        print('\n开始训练模型...')
        print(f'目标验证准确率: {self.target_accuracy:.1%}\n')
    
    def on_train_end(self, logs=None):
        """训练结束时调用"""
        print('\n训练完成！')
    
    def on_epoch_end(self, epoch, logs=None):
        """每个epoch结束时调用"""
        logs = logs or {}
        
        # 提取当前指标
        loss = logs.get('loss', 0)
        accuracy = logs.get('accuracy', 0)
        val_loss = logs.get('val_loss', 0)
        val_accuracy = logs.get('val_accuracy', 0)
        
        # 显示训练进度
        print(f'Epoch {epoch + 1}: '
              f'loss={loss:.4f}, accuracy={accuracy:.4f}, '
              f'val_loss={val_loss:.4f}, val_accuracy={val_accuracy:.4f}')
        
        # 检查是否达到目标准确率
        if val_accuracy >= self.target_accuracy:
            print(f'\n已达到目标准确率 {self.target_accuracy:.1%}，停止训练。')
            self.model.stop_training = True

## 4. 使用内置回调函数

Keras提供了多种常用的内置回调函数，可以直接使用。

In [ ]:
# 创建用于保存模型的目录
checkpoint_dir = 'model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# 配置回调函数列表
callbacks_list = [
    # 自定义回调：监控训练进度
    CustomCallback(target_accuracy=0.98),
    
    # ModelCheckpoint: 保存最佳模型
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model.h5'),
        monitor='val_accuracy',  # 监控验证准确率
        save_best_only=True,     # 仅保存最佳模型
        mode='max',              # 监控指标越大越好
        verbose=1
    ),
    
    # EarlyStopping: 提前终止训练
    keras.callbacks.EarlyStopping(
        monitor='val_loss',      # 监控验证损失
        patience=3,              # 等待3个epoch无改善后停止
        restore_best_weights=True,  # 恢复最佳权重
        verbose=1
    ),
    
    # ReduceLROnPlateau: 动态调整学习率
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',      # 监控验证损失
        factor=0.5,              # 学习率衰减因子
        patience=2,              # 等待2个epoch无改善后降低学习率
        min_lr=1e-6,             # 学习率下限
        verbose=1
    ),
    
    # CSVLogger: 记录训练日志到CSV文件
    keras.callbacks.CSVLogger(
        filename='training_log.csv',
        separator=',',
        append=False
    )
]

## 5. 训练模型并应用回调函数

将配置好的回调函数传递给`fit()`方法，在训练过程中自动执行。

In [ ]:
# 训练模型（使用较少的epoch用于快速测试）
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=3,  # 测试用，实际训练可设为20
    validation_split=0.2,
    callbacks=callbacks_list,
    verbose=0  # 关闭默认输出，使用自定义回调的输出
)

## 6. 加载最佳模型并评估

训练完成后，加载ModelCheckpoint保存的最佳模型进行评估。

In [ ]:
# 加载保存的最佳模型
best_model_path = os.path.join(checkpoint_dir, 'best_model.h5')
if os.path.exists(best_model_path):
    best_model = keras.models.load_model(best_model_path)
    print(f'\n成功加载最佳模型: {best_model_path}')
    
    # 在测试集上评估
    test_loss, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
    print(f'\n测试集评估结果:')
    print(f'Loss: {test_loss:.4f}')
    print(f'Accuracy: {test_accuracy:.4f}')
else:
    print(f'未找到保存的模型: {best_model_path}')

## 7. 分析训练日志

读取CSVLogger保存的训练日志，分析训练过程。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取训练日志
if os.path.exists('training_log.csv'):
    log_df = pd.read_csv('training_log.csv')
    print('\n训练日志:')
    print(log_df)
    
    # 可视化训练过程
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # 损失曲线
    ax1.plot(log_df['epoch'], log_df['loss'], 'b-', label='训练损失')
    ax1.plot(log_df['epoch'], log_df['val_loss'], 'r-', label='验证损失')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('损失曲线')
    ax1.legend()
    ax1.grid(True)
    
    # 准确率曲线
    ax2.plot(log_df['epoch'], log_df['accuracy'], 'b-', label='训练准确率')
    ax2.plot(log_df['epoch'], log_df['val_accuracy'], 'r-', label='验证准确率')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('准确率曲线')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print('未找到训练日志文件')

## 8. 高级回调函数示例

实现一个更复杂的回调函数，展示更多高级特性。

In [ ]:
class AdvancedCallback(keras.callbacks.Callback):
    """
    高级回调函数示例
    
    功能:
    1. 记录每个batch的训练时间
    2. 计算并显示每个epoch的平均batch时间
    3. 监控梯度消失/爆炸问题
    4. 自适应调整batch大小(仅演示概念)
    """
    
    def __init__(self):
        super().__init__()
        self.batch_times = []
        self.epoch_start_time = None
    
    def on_epoch_begin(self, epoch, logs=None):
        """记录epoch开始时间"""
        import time
        self.epoch_start_time = time.time()
        self.batch_times = []
    
    def on_epoch_end(self, epoch, logs=None):
        """分析epoch训练情况"""
        import time
        
        epoch_time = time.time() - self.epoch_start_time
        avg_batch_time = np.mean(self.batch_times) if self.batch_times else 0
        
        print(f'\nEpoch {epoch + 1} 统计:')
        print(f'  总耗时: {epoch_time:.2f}秒')
        print(f'  平均batch耗时: {avg_batch_time:.4f}秒')
        
        # 检查梯度问题（通过损失变化判断）
        if logs:
            loss = logs.get('loss', 0)
            if np.isnan(loss) or np.isinf(loss):
                print('  警告: 检测到梯度爆炸！损失值为NaN或Inf')
                self.model.stop_training = True
    
    def on_batch_begin(self, batch, logs=None):
        """记录batch开始时间"""
        import time
        self.batch_start_time = time.time()
    
    def on_batch_end(self, batch, logs=None):
        """记录batch结束时间"""
        import time
        batch_time = time.time() - self.batch_start_time
        self.batch_times.append(batch_time)

# 演示使用高级回调函数
print('\n演示高级回调函数:')
model_advanced = create_model()

history_advanced = model_advanced.fit(
    x_train, y_train,
    batch_size=128,
    epochs=2,  # 仅训练2个epoch进行演示
    validation_split=0.2,
    callbacks=[AdvancedCallback()],
    verbose=0
)

## 9. 回调函数的执行顺序

理解回调函数的执行顺序对于调试和优化训练流程很重要。

In [ ]:
class OrderDemoCallback(keras.callbacks.Callback):
    """演示回调函数执行顺序"""
    
    def __init__(self, name):
        super().__init__()
        self.name = name
    
    def on_train_begin(self, logs=None):
        print(f'{self.name}: on_train_begin')
    
    def on_epoch_begin(self, epoch, logs=None):
        print(f'{self.name}: on_epoch_begin (epoch {epoch})')
    
    def on_epoch_end(self, epoch, logs=None):
        print(f'{self.name}: on_epoch_end (epoch {epoch})')
    
    def on_train_end(self, logs=None):
        print(f'{self.name}: on_train_end')

# 创建多个回调函数测试执行顺序
print('\n演示回调函数执行顺序:')
model_order = create_model()

# 回调函数按列表顺序执行
callbacks_order = [
    OrderDemoCallback('Callback_1'),
    OrderDemoCallback('Callback_2'),
    OrderDemoCallback('Callback_3')
]

# 仅训练1个epoch演示
history_order = model_order.fit(
    x_train[:1000], y_train[:1000],  # 使用少量数据
    batch_size=128,
    epochs=1,
    callbacks=callbacks_order,
    verbose=0
)

print('\n结论: 回调函数按照列表中的顺序依次执行')

## 10. 清理临时文件

In [ ]:
import shutil

# 清理测试产生的文件和目录
if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
    print(f'已删除目录: {checkpoint_dir}')

if os.path.exists('training_log.csv'):
    os.remove('training_log.csv')
    print('已删除文件: training_log.csv')

print('\n清理完成')

## 总结

### 回调函数的优势

1. **模块化**: 将训练逻辑与监控逻辑分离
2. **可复用**: 一次编写，多处使用
3. **灵活性**: 可以组合多个回调函数
4. **非侵入式**: 不需要修改训练循环代码

### 最佳实践

1. **合理使用内置回调**: 优先使用Keras内置的回调函数
2. **自定义扩展**: 当内置功能不满足需求时再自定义
3. **注意执行顺序**: 回调函数按列表顺序执行，注意依赖关系
4. **避免过度使用**: 过多的回调函数会降低训练速度
5. **日志记录**: 使用CSVLogger或TensorBoard记录详细训练信息

### 常见应用场景

- **实验跟踪**: 记录不同超参数配置的训练结果
- **资源优化**: 在验证集性能饱和时提前终止训练
- **模型选择**: 自动保存验证集上表现最好的模型
- **学习率调度**: 根据训练进度动态调整学习率
- **异常处理**: 检测并处理训练过程中的异常情况